<a href="https://colab.research.google.com/github/MUNEEB68/Machine-learning-task/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import random

# Ensure inline plotting
%matplotlib inline

# Transformations for the data
transform_ = transforms.ToTensor()

# Load CIFAR-10 dataset
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_)

# DataLoader for training and testing data for efficient memory management
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
print("Hi")
# Class for the model
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 16 * 2 * 2)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        X = self.fc4(X)
        return F.log_softmax(X, dim=1)

torch.manual_seed(500)
model = ConvolutionalNetwork()

# Optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training
epochs = 50
losses = []
correct = []

for epoch in range(epochs):
    total_correct = 0
    total_images = 0
    for batch_idx, (X_train, y_train) in enumerate(train_loader):
        batch_idx += 1

        # Forward pass
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        predicted = torch.max(y_pred.data, 1)[1]  # Get the index of the max log-probability
        batch_correct = (predicted == y_train).sum().item()  # Count correct predictions in the batch
        total_correct += batch_correct  # Accumulate total correct predictions
        total_images += y_train.size(0)  # Accumulate total number of images

        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch}  Batch: {batch_idx}  Loss: {loss.item()} ')

    losses.append(loss.item())
    correct.append(total_correct)
    print(f'for Epoch: {epoch}  Accuracy: {total_correct}/{total_images}')

print(correct)




In [ ]:
# Evaluation on the test data
model.eval()  # Set the model to evaluation mode

test_correct = 0
test_total = 0
batch_loss = 0

with torch.no_grad():  # No need to compute gradients during evaluation
    for batch_idx, (X_test, y_test) in enumerate(test_loader):
        y_pred = model(X_test)
        loss = criterion(y_pred, y_test)  # Compute the loss for the batch
        batch_loss += loss.item()  # Accumulate the loss

        predicted = torch.max(y_pred.data, 1)[1]  # Get the index of the max log-probability
        test_correct += (predicted == y_test).sum().item()  # Count correct predictions
        test_total += y_test.size(0)  # Accumulate total number of images

        if (batch_idx + 1) % 10 == 0:
            print(f'Batch {batch_idx + 1}: Loss = {batch_loss / 100:.4f}')
            batch_loss = 0  # Reset the batch loss

test_accuracy = test_correct / test_total * 100
print(f'Test Accuracy: {test_correct}/{test_total} ({test_accuracy:.2f}%)')



In [ ]:
# Class names for CIFAR-10 dataset
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Function to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get a random image from the test dataset
dataiter = iter(test_loader)
images, labels = next(dataiter)  # Correct usage of next()
index = random.randint(0, len(images) - 1)
image, label = images[index], labels[index]

# Show the image
imshow(image)

# Predict the label using the model
model.eval()
with torch.no_grad():
    output = model(image.unsqueeze(0))  # add batch dimension
    _, predicted = torch.max(output, 1)

# Print the true and predicted labels
print(f'True label: {classes[label]}')
print(f'Predicted label: {classes[predicted.item()]}')


NameError: name 'test_loader' is not defined

In [ ]:
# Save the model
torch.save(model.state_dict(), 'cifar10_model.pth')
print("Model saved as cifar10_model.pth")


Model saved as cifar10_model.pth
